# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF data frame called `gdf`.

In [14]:
gdf = cudf.read_csv('./data/week2.csv', usecols = ['lat','long', 'infected'])

## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF data frame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [3]:
all_med = cudf.read_csv('./data/hospitals.csv')
all_med.append(cudf.read_csv('./data/hospitals.csv'))

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.37999725,-0.406042069,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.31513214,-0.556289494,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.43719482,-2.847192764,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.4597435,-2.245468855,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.07812119,-0.030604055,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,10617466,RTH18,Hospital,UNKNOWN,Independent Sector,Visible,False,University Hospital Of Wales,C/O John Radcliffe Hospital,Headley Way,...,<NA>,OX3 9DU,51.76387405,-1.219792008,RTH,Oxford University Hospitals NHS Foundation Trust,<NA>,<NA>,<NA>,<NA>
1225,10617482,RX3KI,Hospital,UNKNOWN,Independent Sector,Visible,False,Foss Park Hospital,Huntington House,Jockey Lane,...,<NA>,YO32 9XW,53.98666382,-1.051122308,RX3,"Tees, Esk and Wear Valleys NHS Foundation Trust",<NA>,<NA>,<NA>,<NA>
1226,10617567,RGT1W,Hospital,UNKNOWN,Independent Sector,Visible,False,Jersey General Hospital,The Parade,<NA>,...,<NA>,JE1 3UH,<NA>,<NA>,RGT,Cambridge University Hospitals NHS Foundation ...,<NA>,<NA>,<NA>,<NA>
1227,10617714,RAX0A,Hospital,UNKNOWN,Independent Sector,Visible,False,London North West University Hospitals,Watford Road,<NA>,...,<NA>,HA1 3UJ,51.57540894,-0.322022736,RAX,Kingston Hospital NHS Foundation Trust,<NA>,<NA>,<NA>,<NA>


Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

In [10]:
all_med.dropna(subset=['Latitude','Longitude'], inplace= True)

## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` data frame you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [16]:
med_lat = cp.asarray(all_med['Latitude'])
med_long = cp.asarray(all_med['Longitude'])
northing, easting = latlong2osgbgrid_cupy(med_lat, med_long)
all_med['northing'], all_med['easting'] = northing, easting

## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [17]:
knn = cuml.NearestNeighbors(n_neighbors = 1)
knn.fit(all_med[['northing','easting']])

NearestNeighbors(n_neighbors=1, verbose=4, handle=<cuml.common.handle.Handle object at 0x7f1c087a81f0>, algorithm='brute', metric='euclidean', p=2, metric_params=None, output_type='cudf')

Save every infected member in `gdf` into a new dataframe called `infected_gdf`.

In [24]:
infected_gdf = gdf.loc[gdf['infected'] == 1]

Create `northing` and `easting` values for `infected_gdf`.

In [29]:
infected_gdf_lat = cp.asarray(infected_gdf['lat'])
infected_gdf_long = cp.asarray(infected_gdf['long'])
northing, easting= latlong2osgbgrid_cupy(infected_gdf_lat,infected_gdf_long)
infected_gdf['northing'], infected_gdf['easting'] = northing, easting

Use `knn.kneighbors` with `n_neighbors=1` on `infected_gdf`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [30]:
distances, indices = knn.kneighbors(infected_gdf[['northing','easting']])

### Check Your Solution

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

In [31]:
indices.head()

0    648
1    648
2    648
3    648
4    650
dtype: int64

Here you can print an infected individual's coordinates from `infected_gdf`:

In [32]:
infected_gdf.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat             53.715826
long            -2.430079
infected         1.000000
norhting    424489.783814
easting     371619.678741
northing    424489.783814
Name: 1346586, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [33]:
all_med.iloc[1234] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

,﻿OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax,northing,easting


<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)